In [39]:
from gensim.test.utils import common_texts, get_tmpfile
from gensim.models import Word2Vec

import time
import os
import numpy as np

import tensorflow as tf

modelfileName = "CS286-Word2Vec.model"
training_size_arr_actual = [0, 0, 0]
training_size_arr_simulated = [0, 0, 0, 0]

dirNameInput = 'Word2VecData_Input'
dirNameOutput = 'Word2VecData_Output'


Word2Vec_model = None

X = None
Y = None
x_train = None
y_train = None
x_test = None
y_test = None

model = None


In [40]:
def createDataFiles():
    import time
    start = time.time()
    stride = 2
    
    fileNamesActual = ['autopark.dat', 'drive.dat', 'idle.dat']
    
    # To only work with simulated data start commenting from here
    fileInput = open("Word2VecData_Input\Word2VecDataActual.txt", "w")
    fileOutput = open("Word2VecData_Output\Word2VecDataActual.txt", "w")
    

    for i in range(len(fileNamesActual)):
        with open(fileNamesActual[i]) as f:
            content = f.readlines()

        # Remove the extra spaces from the sentences
        content = [x.strip() for x in content]
        value = int(0.8 * len(content))
    
        training_size_arr_actual[i] = value
        
        value = len(content)
        print(value)

        cntCntr = 0
        while cntCntr < value:
            iteration = 0
            currMessagePackets = ""

            while iteration < 5 and cntCntr < value:
                parts = content[cntCntr].split(",")

                # this is the whole message
                DATA = parts[3].split(":")[1].replace(" ", "")

                currMessagePackets += str(DATA) + " "

                cntCntr += 1
                iteration += 1

            if cntCntr < value:
                cntCntr -= stride
            else:
                break
            fileInput.write(currMessagePackets + "\n")
            fileOutput.write(str(i) + "\n")
    fileInput.close()
    fileOutput.close()
    # To only work with Simulated data end commenting till here

    # To only work with actual data start commenting from here
    # For Simulated data
    fileInput = open("Word2VecData_Input\Word2VecDataSimulated.txt", "w")
    fileOutput = open("Word2VecData_Output\Word2VecDataSimulated.txt", "w")
    fileNamesSimulated = ['New_DRIVE_Data.txt', 'New_IDLE_Data.txt', 'DRIVE.rtf', 'IDLE.rtf']
    arrSimulated = [1, 2, 1, 2]

    for i in range(len(fileNamesSimulated)):
        with open(fileNamesSimulated[i]) as f:
            content = f.readlines()

        # Remove the extra spaces from the sentences
        content = [x.strip() for x in content]
        
        training_size_arr_simulated[i] = (0.8 * len(content))
        
        print(len(content))

        cntCntr = 0
        while cntCntr < len(content):
            iteration = 0
            currMessagePackets = ""

            while iteration < 5 and cntCntr < len(content):
                # Remove the noise from the files
                DATA = content[cntCntr][15:38]

                # this is the whole message
                parts = DATA.split()

                tempo = ""
                for j in range(8):
                    if j >= len(parts) or parts[j] == "  ":
                        tempo += "00"
                    else:
                        tempo += parts[j]

                currMessagePackets += str(tempo) + " "
                cntCntr += 1
                iteration += 1

            if cntCntr < len(content):
                cntCntr -= stride
            else:
                break
            fileInput.write(currMessagePackets + "\n")
            fileOutput.write(str(arrSimulated[i]) + "\n")

    fileInput.close()
    # To only work with actual data end commenting till here
    end = time.time()

    print("Awesome !!! File processing done !!!")
    print("Total Time for file processing ---> ", end - start)


In [41]:
class IteratingClass:
    def __init__(self, dirName):
        self.dirName = dirName

    def __iter__(self):
        for fName in os.listdir(self.dirName):
            for line in open(os.path.join(self.dirName, fName)):
                yield line.split()

def createWord2VecModels():
    start = time.time()
    dataDirec = IteratingClass(dirNameInput)
    model = Word2Vec(dataDirec, size=200, window=5, min_count=1, workers=8)
    model.save(modelfileName)
    end = time.time()
    print("\n\nTraining Successful for Word2Vec Model!!!")
    print("Total Time for Word2Vec model -> ", (end - start))


In [42]:
def train():
    createDataFiles()
    createWord2VecModels()



In [43]:
train()

5592
5582
5601
29977
29993
1814
1802
Awesome !!! File processing done !!!
Total Time for file processing --->  1.6504642963409424


Training Successful for Word2Vec Model!!!
Total Time for Word2Vec model ->  2.8183348178863525


In [44]:
# Convert the data into Numpy Arrays
start = time.time()
Word2Vec_model = Word2Vec.load(modelfileName)
x_train = None
def getNumpyArray():
    global X
    global Y
    global dirNameInput
    X = None
    Y = None
    print("Processing started ")
    result = []
    for fName in os.listdir(dirNameInput):
        for line in open(os.path.join(dirNameInput, fName)):
            parts = line.split()
            for i in range(0, len(parts)):
                result.append(Word2Vec_model.wv[parts[i]])
    X = np.array(result)
                
                
    print("Processing started for output ")
    resultY = []
    for fName in os.listdir(dirNameOutput):
        for line in open(os.path.join(dirNameOutput, fName)):
            currV = int(line)
            for i in range(0,5):
                resultY.append(currV)
    
    Y = np.array(resultY)
    print(X.shape)    
    print(Y.shape)    
    end = time.time()
    print("total time -> ", end -start)





#    LSTM

In [45]:
getNumpyArray()



Processing started 
Processing started for output 
(133885, 200)
(133885,)
total time ->  8.131474494934082


In [46]:
#execute LSTM model

def executeModel(eph):
    start = time.time()
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.LSTM(5, return_sequences=True,input_shape=(200,1)))
    model.add(tf.keras.layers.LSTM(5 ,input_shape=(200,1)))
    model.add(tf.keras.layers.Dense(1, activation='relu'))


    print ('Compiling...')
    model.compile(loss='binary_crossentropy',optimizer='rmsprop',metrics=['accuracy'])
    model.fit(X_train, y_train, batch_size = 30 , epochs=eph, verbose = 1)

    val_loss, val_acc = model.evaluate(X_test, y_test)
    print("Test score: ", val_loss)
    print("Test accuracy: ", val_acc)

    end = time.time()
    print("Total Time -> ", (end - start))




In [47]:
#split the data and train the model
X = X[:10000]
Y = Y[:10000]
X = X.reshape(10000,200,1)
print(X.shape)
print(Y.shape)

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.30, random_state=42)
executeModel(10)

(10000, 200, 1)
(10000,)
Compiling...
Epoch 1/10
7000/7000 [==============================] - 114s 16ms/step - loss: 0.7439 - acc: 0.7089
Epoch 2/10
7000/7000 [==============================] - 86s 12ms/step - loss: 1.1921e-07 - acc: 1.0000
Epoch 3/10
7000/7000 [==============================] - 91s 13ms/step - loss: 1.1921e-07 - acc: 1.0000
Epoch 4/10
7000/7000 [==============================] - 105s 15ms/step - loss: 1.1921e-07 - acc: 1.0000
Epoch 5/10
7000/7000 [==============================] - 107s 15ms/step - loss: 1.1921e-07 - acc: 1.0000
Epoch 6/10
7000/7000 [==============================] - 105s 15ms/step - loss: 1.1921e-07 - acc: 1.0000
Epoch 7/10
7000/7000 [==============================] - 88s 13ms/step - loss: 1.1921e-07 - acc: 1.0000
Epoch 8/10
7000/7000 [==============================] - 91s 13ms/step - loss: 1.1921e-07 - acc: 1.0000
Epoch 9/10
7000/7000 [==============================] - 89s 13ms/step - loss: 1.1921e-07 - acc: 1.0000
Epoch 10/10
3000/3000 [============

In [ ]:
def evaluateWordToVec():
    Word2Vec_model = Word2Vec.load(modelfileName)
    print("Loaded successfully")
    #print(model.wv['FEB0FF999E500000'])
    print(len(Word2Vec_model.wv['FEB0FF999E500000']))

    #print(model.wv['000200000000002A'])
    print(len(Word2Vec_model.wv['000200000000002A']))
    
    #print(model.wv['FEB0FF999E500000'] + model.wv['000200000000002A'])
    
    v = Word2Vec_model.wv['FEB0FF999E500000'] + Word2Vec_model.wv['000200000000002A']
    print(type(v))
    #print(v/2)
evaluateWordToVec()